# Capstone – Predictive Model + Traffic Mapping for Riyadh

This notebook includes:

1. Loading reference congestion charge impact data (from global cities)
2. Predictive model (linear + gradient boosting) for Riyadh after congestion charge
3. Simple visualizations of traffic volume, travel time, safety improvements
4. TomTom Map integration for Riyadh traffic overlay + predictive impact markers

In [ ]:
## 1. Load Global Impact Data (pre/post charge)

"""
Each row = a city case study, with before and after values:
- traffic_volume_change_pct
- travel_time_change_pct
- accident_change_pct

We will use this to train a model to predict Riyadh’s expected changes.
"""

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

data = pd.DataFrame({
    'city': ['London', 'Singapore', 'Milan', 'Dubai'],
    'traffic_volume_change_pct': [-15, -16, -28, -25],
    'travel_time_change_pct': [-30, -35, -25, -50],
    'accident_change_pct': [-40, -20, -26, -10],
    'cars_before': [300_000, 250_000, 400_000, 500_000],
    'density': [9000, 7000, 8000, 4000]  # people/km²
})

data

---

## 2. Train Predictive Model

"""
We'll use simple linear regression and gradient boosting regressors
to learn from global data and predict Riyadh outcomes

Features: cars_before, density
Targets: change in traffic volume, travel time, and accident
"""

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

X = data[['cars_before', 'density']]
y_traffic = data['traffic_volume_change_pct']
y_time = data['travel_time_change_pct']
y_safety = data['accident_change_pct']

lin_model = LinearRegression().fit(X, y_traffic)
gbr_model = GradientBoostingRegressor().fit(X, y_time)

# Riyadh baseline input
riyadh_input = pd.DataFrame({
    'cars_before': [600_000],
    'density': [5000]
})

pred_traffic = lin_model.predict(riyadh_input)[0]
pred_time = gbr_model.predict(riyadh_input)[0]
pred_safety = lin_model.predict(riyadh_input)[0] * 0.6  # lower elasticity

print(f"Predicted traffic drop: {pred_traffic:.1f}%")
print(f"Predicted travel time improvement: {pred_time:.1f}%")
print(f"Predicted accident reduction: {pred_safety:.1f}%")

---

## 3. Plot Predictions

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
labels = ['Traffic Volume', 'Travel Time', 'Accidents']
values = [pred_traffic, pred_time, pred_safety]
colors = ['#d62728', '#2ca02c', '#1f77b4']

plt.bar(labels, values, color=colors)
plt.title("Predicted Impact of Congestion Charge – Riyadh")
plt.ylabel("% Change")
plt.axhline(0, color='gray')
plt.grid(axis='y')
plt.tight_layout()
plt.show()

---

## 4. Display Map with TomTom Traffic Overlay

"""
This section loads an HTML map centered on Riyadh with traffic overlays
- Requires TomTom API Key stored in .env or as TOMTOM_API_KEY
- Shows live traffic on Riyadh
"""

from IPython.display import HTML
import os

tomtom_key = os.getenv("TOMTOM_API_KEY") or "YOUR_TOMTOM_API_KEY"  # Replace if no .env

iframe_html = f"""
<iframe
  width=\"100%\"
  height=\"500\"
  src=\"https://api.tomtom.com/map/1/staticimage?layer=basic&style=main&format=png&traffic=true&zoom=12&center=46.6753,24.7136&key={tomtom_key}\">
</iframe>
"""

HTML(iframe_html)

---

## Notes:

- Replace `YOUR_TOMTOM_API_KEY` with your key or set an `.env` file with `TOMTOM_API_KEY`
- This setup is compatible with VS Code and will render charts and map inline
- You can expand by integrating incident data or simulating traffic zone overlays